In [ ]:
# Imports

import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%tensorflow_version 1.x
import os
import shutil # high-level operations on files
from tqdm import tqdm # Progress bar and status logging
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix

import cv2 # computer vision algorithms

# Importing the Keras libraries and packages
from keras import utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout


TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Configuration

DATASET_DIR = 'C:\\Users\\yash\\Desktop\\kaggle\\real-and-fake-face-detection\\real_and_fake_face_detection/real_and_fake_face'
TRAIN_DIR = 'C:\\Users\\yash\\Desktop\\kaggle\\train_dataset'
TEST_DIR = 'C:\\Users\\yash\\Desktop\\kaggle\\test_dataset'

RATE = 0.2 # splitting proportion for training and test datasets

# Parameters for Grid Search

N_EPOCHS = [1] #[20, 40, 100, 200]
OPTIMIZERS = ['adam'] #['adam', 'rmsprop', 'SGD']
DROPOUT_RATES =  [0.4]
LOSS_FUNCTIONS = ['binary_crossentropy']  #['sparse_categorical_crossentropy', 'kullback_leibler_divergence']    

In [ ]:
os.mkdir(TRAIN_DIR)
os.mkdir(TRAIN_DIR+'\\fake')
os.mkdir(TRAIN_DIR+'\\real')

os.mkdir(TEST_DIR)
os.mkdir(TEST_DIR+'\\fake')
os.mkdir(TEST_DIR+'\\real')

Updated folder structure:

In [ ]:
for dirname, _, filenames in os.walk('C:\\Users\\yash\\Desktop\\kaggle'):
    print (dirname)

In [ ]:
# Split folders with images into training, validation and test folders.
# OPTION 1 (using split-folders)

#pip install split-folders

#import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
#split_folders.ratio('input_folder', output="output", seed=1337, ratio=(.8, .1, .1)) # default values


In [ ]:
# Split image files into test and training set 
# OPTION 2 (copying files into newly created folders)
files_real = os.listdir(f'{DATASET_DIR}\\training_real')
files_fake = os.listdir(f'{DATASET_DIR}\\training_fake')


# sample from each class to create a test set
np.random.seed(0)
files_real_test = np.random.choice(
    files_real,
    size=round(len(files_real) * RATE),
    replace=False,
    p=None)

files_real_train = list(set(files_real) - set(files_real_test)) #[file for file in files_real if file not in files_real_test] 

files_fake_test = np.random.choice(
    files_fake,
    size=round(len(files_fake) * RATE),
    replace=False,
    p=None)

files_fake_train = list(set(files_fake) - set(files_fake_test)) #[file for file in files_fake if file not in files_fake_test] 

for file in files_real_train:
    shutil.copyfile(DATASET_DIR+'/training_real/'+file, TRAIN_DIR+'/real/'+file) 

for file in files_fake_train:
    shutil.copyfile(DATASET_DIR+'/training_fake/'+file, TRAIN_DIR+'/fake/'+file) 

for file in files_real_test:
    shutil.copyfile(DATASET_DIR+'/training_real/'+file, TEST_DIR+'/real/'+file) 

for file in files_fake_test:
    shutil.copyfile(DATASET_DIR+'/training_fake/'+file, TEST_DIR+'/fake/'+file) 
    

FileNotFoundError: ignored

In [ ]:
train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
test_samples = sum([len(files) for r, d, files in os.walk(TEST_DIR)])
print('Number of training images: {} \nNumber of test images: {}'.format(train_samples, test_samples))

Number of training images: 0 
Number of test images: 0


In [ ]:
# load and show an image with Pillow
# from PIL import Image
# image = Image.open('/kaggle/test_dataset/fake/hard_39_1111.jpg')
# # some details about the image
# print(image.format)
# print(image.mode)
# print(image.size)

In [ ]:

def get_images(path, img_shape=(64, 64)):
 
    '''
    Returns a np array of images and labels from path
    Images must be stored in path/class1, path/class2
    '''
    main_path = path
    y = []
    list = [name for name in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, name))]
    print(list)
    image_collection = []
    for idx,folder in enumerate(list):
 
        label = idx
        
        sub_list = sorted(os.listdir(os.path.join(main_path,folder)))
 
        for i in tqdm(range(1, len(sub_list))):
            image_path = os.path.join(main_path, folder, sub_list[i])
            read_image = cv2.imread(image_path)
            image_resized = cv2.resize(read_image, img_shape, interpolation=cv2.INTER_AREA)
 
            image = np.float32(image_resized)
            image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) #Change alpha, beta according to the preprocessing you desire
            
            image_collection.append(image)
            
            y.append(label)
 
    y = np.array(y)
    y = utils.to_categorical(y,num_classes=len(list))
 
    return image_collection, y[:,0] 


In [ ]:
# Preparing test and trainng datasets
X_train,y_train = get_images("/content/drive/My Drive/kaggle/train_dataset",img_shape=(64,64))
#X_test,y_test = get_images("/content/drive/My Drive/kaggle/test_dataset",img_shape=(64,64))
#dict_data1 = np.load('/content/drive/My Drive/kaggle/xtrain.npz')
dict_data2 = np.load('/content/drive/My Drive/Colab Notebooks/xtest.npz')
dict_data3 = np.load('/content/drive/My Drive/Colab Notebooks/ytrain.npz')
dict_data4 = np.load('/content/drive/My Drive/Colab Notebooks/ytest.npz')
#X_train = dict_data1['arr_0']
X_test = dict_data2['arr_0']
y_train = dict_data3['arr_0']
y_test = dict_data4['arr_0']
X_train = np.array(X_train)
#X_test = np.array(X_test)
#np.savez_compressed('xtest.npz',X_test)
np.savez_compressed('xtrain.npz',X_train)
#np.savez_compressed('ytest.npz',y_test)
#np.savez_compressed('ytrain.npz',y_train)

 

  1%|          | 6/767 [00:00<00:12, 59.99it/s]

['fake', 'real']


100%|██████████| 864/864 [04:01<00:00,  3.57it/s]


In [ ]:
print('Training set', X_train.shape)
print('Test set', X_test.shape)

Training set (1631, 64, 64, 3)
Test set (406, 64, 64, 3)


We don't have too much data to train the network. 
One of possible workarounds is to use ImageDataGenerator.
On the one hand, it does allow us to generate additional examples. On the other hand, all of these examples are based on a too small dataset and the network still cannot generalize to data it was never trained on 


In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
# train_datagen = ImageDataGenerator(shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)
# batches = 0
# for _ in train_datagen.flow_from_directory(directory=TRAIN_DIR, class_mode='binary', batch_size = 32, save_prefix='real_aug', save_to_dir=TRAIN_DIR+'/real', save_format='jpeg'):
#     batches += 1
#     if batches >= number_of_real_images / 32:
#         break # to stop the indefinite generator loop





In [ ]:
#Shuffle training examples
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
#def build_classifier(optimizer, dropout, loss):
classifier = Sequential() # Initialising the CNN    
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu')) 
classifier.add(MaxPooling2D(pool_size = (2, 2))) 
classifier.add(Dropout(0.4))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))  
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.4))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))  
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.4))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid')) #'tanh'))

#from keras.utils.vis_utils import plot_model
#plot_model(classifier, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
classifier.summary()
 


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 32)       

In [ ]:
#preds = classifier.evaluate(x=X_test,y=y_test)
### END CODE HERE ###
print()
#print ("Loss = " + str(preds[0]))
#print ("Test Accuracy = " + str(preds[1]))    
#    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

grid_parameters = {'epochs': N_EPOCHS,
                  'optimizer': OPTIMIZERS,
                  'dropout': DROPOUT_RATES,                  
                  'loss':LOSS_FUNCTIONS                        
                  }


grid_search = GridSearchCV(estimator = classifier,
                           param_grid = grid_parameters,
                           scoring = 'accuracy',
                           cv = 10)








In [ ]:
grid_search = grid_search.fit(X_train, y_train, verbose=0)